# Analytical computation of the Maximum formation efficiency

As described in section 2 from the paper


In [3]:
import numpy as np
import os 
import sys
import pandas as pd
import h5py as h5
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib as mpl
import matplotlib.lines as mlines
import multiprocessing as mp

# add run_data path to sys
sys.path.append('./run_data')
from definitions import sim_flags_dict


######################################
## PLOT setttings
plt.rc('font', family='serif')
from matplotlib import rc
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
fsize, SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE = 30,20,25,30
for obj in ['axes','xtick','ytick']:
    plt.rc(obj, labelsize=SMALL_SIZE)          # controls default text sizes
for obj in ['figure','axes']:
    plt.rc(obj, titlesize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize



home_dir    = os.path.expanduser("~") 
compas_v    = "v03.01.02" #"v02.46.01" # #"#v02.35.02/"
datar_root  =  f"{home_dir}/ceph/CompasOutput/{compas_v}/"


# Compute the average SF mass per system

We use the 'total mass evolved per Z' function that we also use for the yield calculations for this

In [37]:

def totalMassEvolvedPerZ(pathCOMPASh5, x2=0.08, x3=0.5, a1=-0.3, a2=-1.3, a3=-2.3, C1=1.,
                         binaryFraction=0.7, Mmin_universe=0.1, Mmax_universe=300., sampleSize=2000000):
    """_summary_

    Args:
        # COMPAS simulation parameters
        pathCOMPASh5 (_type_, optional): path to your COMPAS file. Defaults to None.

        # Broken powerlaw (Kroupa IMF) parameters
        x1, x2, x3, x4: float, the break points (mass ranges) for the three segments
        a1, a2, a3: float, the power law indices 
        <0.01 - 0.08> a = -0.3, <0.08 - 0.5> a = -1.3, <0.5 - 200> a = -2.3
        C1: float, the normalization constant for the first segment
        
        # Believes about star formation in the Universe
        binaryFraction (int, optional): What fraction of stars are in binaries. Default= 1.
        Mmin_universe, Mmax_universe (float): the min and max mass that stars in the Universe can be born with  Defaults: 0.01 and 200.

    Returns:
        _type_: _description_
    """ 
    x1 = Mmin_universe
    x4 = Mmax_universe

    # Open the COMPAS file
    COMPASdataf = h5.File(pathCOMPASh5, 'r')

    # Min and max M sampled in your COMPAS simulation.
    COMPAS_m1       = COMPASdataf['BSE_System_Parameters']['Mass@ZAMS(1)'][()]
    Mlower_COMPAS   = np.min(COMPAS_m1)
    Mupper_COMPAS   = np.max(COMPAS_m1)

    ##########################
    # Create Sample Universe 
    ##########################
    # we will use 'inverse transform sampling method' to sample our sample Universe from the IMF

    ### Primary mass
    # first we compute the y-values of the CDF of our IMF at Mmin_universe and Mmax_universe
    # Mmin_universe and Mmax_universe have to be between x1 and x4
    CDFmin = CDFbrokenPowerLaw(np.array([Mmin_universe]), x1, x2, x3, x4, a1, a2, a3, C1)
    CDFmax = CDFbrokenPowerLaw(np.array([Mmax_universe]), x1, x2, x3, x4, a1, a2, a3, C1)

    # Now we can sample Uniformly from the CDF between CDFmin and CDFmax
    drawM1      = np.random.uniform(CDFmin,CDFmax,sampleSize)
    # Convert CDF values back to masses
    M1          = invertCDFbrokenPowerLaw(drawM1, x1, x2, x3, x4, a1, a2, a3, C1)

    ### Binary fraction
    # we want that binaryFraction of the stars are in binaries
    # Hence by drawing between 0-1, we have to throw out everything that is above binaryFraction (i.e. = single and m2 = 0)
    # ! NOTE that this assumes that the binary Fraction is mass indepent! > Future work to implenet Max Moe ps and qs options
    drawBinary      = np.random.uniform(0,1,sampleSize)
    maskBinary      = drawBinary < binaryFraction  #booleans

    ### Secondary mass
    # mass ratio (q = m2/m1) distribution is assumed to be flat 
    # so then the drawM2 (if it is in a binary) just becomes the mass fraction.
    drawM2          = np.random.uniform(0,1,sampleSize)    # we are actually sampling q
    M2              = np.zeros(sampleSize)                 #
    M2[maskBinary]  = drawM2[maskBinary] * M1[maskBinary]  # = q * m1, all the ones outside the mask remain zero
    
    totalMassInStarFormation = np.sum(M1) + np.sum(M2)

    ##########################
    # Select what lies in the range of COMPAS
    ##########################
    # mask M1 and M2 to see what lies in the range of COMPAS
    maskM1          = (M1>=Mlower_COMPAS) & (M1<=Mupper_COMPAS)
    maskBinaries    = (M2!=0)
    mask_COMPAS     = maskM1 & maskBinaries

    totalMassEvolvedCOMPAS = np.sum(M1[mask_COMPAS]) + np.sum(M2[mask_COMPAS])

    ##########################
    # Finally compute the tot mass evolved per Z
    ##########################
    
    # load a bit more COMPAS data
    COMPAS_m2       = COMPASdataf['BSE_System_Parameters']['Mass@ZAMS(2)'][()]
    COMPAS_metals   = COMPASdataf['BSE_System_Parameters']['Metallicity@ZAMS(1)'][()]
    uniqueZ_COMPAS  = np.unique(COMPAS_metals)
    
    # Determine if your samples are weighted
    # boolWeighted = 'mixture_weight' in COMPASdataf['BSE_System_Parameters'].keys()

    # I assume that if you have more than 100 metallicities, it's not discrete, but a continuous Z distribution
    w_NbinariesEvolvedPerZ = []                                                           # Nbinaries simulated per Z //floor
    for Z in uniqueZ_COMPAS:
        mask = COMPAS_metals == Z
        Nbinaries = len(COMPAS_m1[mask])
        w_NbinariesEvolvedPerZ.append(Nbinaries)
    w_NbinariesEvolvedPerZ        = np.array(w_NbinariesEvolvedPerZ)
    w_AverageMassPerBinaryCOMPAS  = totalMassEvolvedCOMPAS / len(M1[mask_COMPAS])         # average mass of a binary in COMPAS simulation  //floor
    w_MassEvolvedPerZ             = w_AverageMassPerBinaryCOMPAS * w_NbinariesEvolvedPerZ     # //floor    

    # Simulation with discrete metallicities
    total = []
    for Z in uniqueZ_COMPAS:
        Zmask = COMPAS_metals == Z
        total.append( np.sum(COMPAS_m1[Zmask]) + np.sum(COMPAS_m2[Zmask]) )

        MassEvolvedPerZ  = np.array(total)

    # fraction of total universe that was sampled by COMPAS
    fraction = totalMassEvolvedCOMPAS/float(totalMassInStarFormation)

    # We need to muliply the mass evolved per metallicity times (1/fraction) to know the total mass evolved per metallicity
    totalMassEvolvedPerMetallicity = (MassEvolvedPerZ)/(fraction)

    return w_NbinariesEvolvedPerZ, w_AverageMassPerBinaryCOMPAS, w_MassEvolvedPerZ, totalMassEvolvedCOMPAS, float(totalMassInStarFormation),  MassEvolvedPerZ


def CDFbrokenPowerLaw(x, x1=0.01, x2=0.08, x3=0.5, x4=200, a1=-0.3, a2=-1.3, a3=-2.3, C1=1):
    """
    CDF values of a three-part broken powerlaw representing a Kroupa IMF by default.
    
    Parameters:
    x: array-like, the input values
    x1, x2, x3, x4: float, the break points (mass ranges) for the three segments
    a1, a2, a3: float, the power law indices 
    C1: float, the normalization constant for the first segment
    
    Returns:
    yvalues: array-like, the output values of the CDF
    """
    
    # Initialize the output array
    yvalues = np.zeros(len(x))
    
    # Calculate the normalization constants for the other segments
    # Ensuring that the next segments start where the previous segment ends
    C2 = float(C1 * (x2**(a1-a2)))
    C3 = float(C2 * (x3**(a2-a3)))
    
    # Calculate the normalization factors for the three segments
    N1 = float(((1./(a1+1)) * C1 * (x2**(a1+1))) - ((1./(a1+1)) * C1 * (x1**(a1+1))))
    N2 = float(((1./(a2+1)) * C2 * (x3**(a2+1))) - ((1./(a2+1)) * C2 * (x2**(a2+1))))
    N3 = float(((1./(a3+1)) * C3 * (x4**(a3+1))) - ((1./(a3+1)) * C3 * (x3**(a3+1))))
    
    # Calculate the denominator of the CDF
    bottom = N1+N2+N3
    
    # Calculate the CDF values for x range: x1<=x<x2
    mask1 = (x>=x1) & (x<x2)
    top1 = ( (1./(a1+1) ) * C1 * (x[mask1]**(a1+1) ) - (1./(a1+1) ) * C1 * (x1**(a1+1) ) ) 
    yvalues[mask1] = top1/bottom
    
    # Calculate the CDF values for x range: x2<=x<x3
    mask2 = (x>=x2) & (x<x3)
    top2 =  N1 + ( (1./(a2+1) ) * C2 * (x[mask2]**(a2+1) ) - (1./(a2+1)) * C2 * (x2**(a2+1) ) ) 
    yvalues[mask2] = top2/bottom
    
    # Calculate the CDF values for x range: x3<=x<=x4
    mask3 = (x>=x3) & (x<=x4)
    top3 =  N1 + N2 + ( (1./(a3+1)) * C3 * (x[mask3]**(a3+1)) - (1./(a3+1)) * C3 * (x3**(a3+1) ) )
    yvalues[mask3] = top3/bottom
    
    return yvalues


def invertCDFbrokenPowerLaw(CDF, x1, x2, x3, x4, a1, a2, a3, C1):
    """
    Invert y-values of a CDF back to x-vals (i.e. the masses)
    Specifically for a three-part piece-wise powerlaw representing a Kroupa IMF by default. 

    Parameters:
    CDF: array-like, the CDF values to invert
    x1, x2, x3, x4: float, the break points (ranges) for the three segments
    a1, a2, a3: float, the power law indices for the three segments
    C1: float, the normalization constant for the first segment

    Returns:
    xvalues: array-like, the inverted CDF values
    """
    
    # Calculate the normalization constants for the second and third segments
    C2 = float(C1 * (x2**(a1-a2)))
    C3 = float(C2 * (x3**(a2-a3)))
    
    # Calculate the area under the curve for each segment
    N1 = float(((1./(a1+1)) * C1 * (x2**(a1+1))) - ((1./(a1+1)) * C1 * (x1**(a1+1))))
    N2 = float(((1./(a2+1)) * C2 * (x3**(a2+1))) - ((1./(a2+1)) * C2 * (x2**(a2+1))))
    N3 = float(((1./(a3+1)) * C3 * (x4**(a3+1))) - ((1./(a3+1)) * C3 * (x3**(a3+1))))
    
    # Calculate the CDF values at the breakpoints
    CDFx2 = CDFbrokenPowerLaw(np.array([x2,x2]), x1, x2, x3, x4, a1, a2, a3, C1)[0]
    CDFx3 = CDFbrokenPowerLaw(np.array([x3,x3]), x1, x2, x3, x4, a1, a2, a3, C1)[0]

    # Initialize the output array
    xvalues = np.zeros(len(CDF))
    
    # Calculate the inverse CDF values for the first segment
    mask1 = (CDF < CDFx2)
    xvalues[mask1] =  (((CDF[mask1]*(N1+N2+N3))  + \
                      ( (1./(a1+1))*C1*(x1**(a1+1))))/((1./(a1+1))*C1))**(1./(a1+1))
    
    # Calculate the inverse CDF values for the second segment
    mask2 = (CDFx2<= CDF) & (CDF < CDFx3)
    xvalues[mask2] = ((((CDF[mask2]*(N1+N2+N3))-(N1))  + \
                      ( (1./(a2+1))*C2*(x2**(a2+1))))/((1./(a2+1))*C2))**(1./(a2+1))
    
    # Calculate the inverse CDF values for the third segment
    mask3 = (CDFx3<= CDF) 
    xvalues[mask3] = ((((CDF[mask3]*(N1+N2+N3))-(N1+N2))  + \
                      ((1./(a3+1))*C3*(x3**(a3+1))))/((1./(a3+1))*C3))**(1./(a3+1))
    
    # Return the inverse CDF values
    return xvalues


In [38]:
sim_name = 'NewWinds_RemFryer2012'

Sampe_size = int(2e6)

w_NbinariesEvolvedPerZ, w_AverageMassPerBinaryCOMPAS, w_MassEvolvedPerZ, totalMassEvolvedCOMPAS, totalMassInStarFormation,  MassEvolvedPerZ = totalMassEvolvedPerZ(f'{datar_root}/{sim_name}/COMPAS_Output_combinedZ.h5', sampleSize=Sampe_size)




In [49]:
average_mass_per_system_univ = totalMassInStarFormation/Sampe_size
print(f'Average mass per system in Universe {average_mass_per_system_univ}' )

Average mass per system in Universe 0.899409810483058


#

\begin{equation}
\frac{dP}{dm} = m^{-2.35}
 \end{equation}

\begin{equation}
f_{primary} = \frac{m^{-1/35}|_{Mmin}^{300}}{m^{-1.35}|_{0.1}^{300}} = \frac{ 300^{-1.35} - Mmin^{-1.35} }{-22.39}
\end{equation}

 for BBHs, $Mmin = 20$, for NS, $Mmin = 8$


In [46]:
f_prim_bbh = (300**-1.35 - 20**-1.35)/(300**-1.35 - 0.1**-1.35)
print(f'f_primary for BBH = {f_prim_bbh}')
f_prim_nsns = (300**-1.35 - 8**-1.35)/(300**-1.35 - 0.1**-1.35)
print(f'f_primary for NSNS {f_prim_nsns}' )

f_primary for BBH = 0.0007625160671152889
f_primary for NSNS 0.002676503668045551


We have a flat in q distribution, so for both BBH and NSNS, $f_{secondary} = 0.5$


In [41]:
f_secondary_bbh = 0.5
f_secondary_nsns = 0.5

for $f_{init sep}$, we adopt the fraction of systems that interacts ever. 

We assume that binaries can form with separations between 0.01AU and 1000 AU 

we assume a flat-in-log distribution of initial separations

\begin{equation}
P(a_i) = 1/a_i
\end{equation}

\begin{equation}
f_{init sep} = \frac{\log(a_i)|_{mina}^{maxa} }{\log(a_i)|_{0.01 AU}^{1000 AU} }
\end{equation}

where $mina$ and $maxa$ are the min and max separation for interaction,
For the minimum, we look at our case A,B C plots, and stars have radii of $3-20R_{\odot}$ at birth (roughly), leading to a range of 0.0279AU - 0.186AU. We adopt the average of $mina \approx 0.1 AU$

For the upper end, we use our max R per Z to estimate this. Very roughly stars range between a max R of 1000 and 5000, so we adopt $maxa \approx 3000 R_{\odot} = 13.95 AU$


\begin{equation}
f_{init sep} = \frac{\log(13.95) - \log(0.1)}{\log(100) - \log(0.01)} \approx 0.42
\end{equation}


In [42]:
f_init_sep = 0.42

 We assume no kicks for BBH so $f_{SN1} = f_{SN2} = 1$

 We assume full kicks for NSNS so $f_{SN1} = f_{SN2} = 0.14$ (See Soumendra's derivation)

In [28]:
f_sn1_bbh = 1.
f_sn2_bbh = 1.


f_sn1_nsns = 0.14
f_sn2_nsns = 0.14

In [57]:

eta_BBH = 1./average_mass_per_system_univ * (f_prim_bbh * f_secondary_bbh * f_init_sep * f_sn1_bbh * f_sn2_bbh )
print(f'eta_BBH = {eta_BBH}')


eta_NSNS = 1./average_mass_per_system_univ * (f_prim_nsns * f_secondary_nsns * f_init_sep * f_sn1_nsns * f_sn2_nsns )
print(f'eta_nsns = {eta_NSNS}')

eta_BBH = 0.0001780371664038314
eta_nsns = 1.2248575642908227e-05


#### for BHNS

$f_{init sep} = 0.42$ stays the same
$f_{SN1} = 1$
$f_{SN2} = 0.14$
$f_{primary} = 7.6 \cdot 10^{-4}$ (same as for BBH)

In [61]:
f_prim_bhns = (300**-1.35 - 20**-1.35)/(300**-1.35 - 0.1**-1.35)
f_sn1_bhns = 1
f_sn2_bhns = 0.14


But now $f_{secondary}$ is a more complicated conditional probability
\begin{equation}
f_{secondary} = P(8 < m_2 < 20\, | m_1 > 20) = \frac{\int_{20}^{300} P(8 < m_2 < 20\, | m_1)P(m1)dm }{P(m1>20) }
\end{equation}

$q = m_2/m_1$ is a flat distribution

we can re-write $m_2$ in terms of q and $m_1$
\begin{equation}
P(8 < m_2 < 20\, | m_1) = P(8/m_1 < q < 20/m_1 | m_1)
\end{equation}

Because q is flat the probability for q is just the difference
\begin{equation}
 P(8/m_1 < q < 20/m_1 | m_1) = \frac{20}{m_1} - \frac{8}{m_1} = \frac{12}{m_1}
\end{equation}

multiplying this by the $m_1$ probability:
\begin{equation}
P(8 < m_2 < 20\, | m_1)P(m1) =\frac{12}{m_1} P(m1)  = \frac{12}{m_1} m_1^{-2.3}
\end{equation}

so we get:
\begin{equation}
f_{secondary} =  \frac{ \int_{20}^{300} 12 m_1^{-3.3} }{ \int_{20}^{300} m_1^{-2.3} }  = \frac{12 \cdot (300^{-2.35} - 20^{-2.35})/-2.35  }{(300^{-1.35} - 20^{-1.35})/-1.35}
\end{equation}

In [62]:
f_secondary_bhns =  12 *  ((300**-2.35 - 20**-2.35)/-2.35) / ((300**-1.35 - 20**-1.35)/(-1.35))
print(f'f_secondary for BHNS {f_secondary_bhns}' )




f_secondary for BHNS 0.3532138172332844


In [65]:
eta_BHNS = 1./average_mass_per_system_univ * (f_prim_bhns * f_secondary_bhns * f_init_sep * f_sn1_bhns * f_sn2_bhns )
print(f'eta_BHNS = {eta_BHNS}')

eta_BHNS = 1.760785240337053e-05
